In [4]:
## Imports 
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Load Data
train = pd.read_csv('data/reduced_new_train.csv', index_col='id')
test = pd.read_csv('data/test.csv', index_col='id')

train = pd.get_dummies(train)
test = pd.get_dummies(test)

# append a column of zeros to test data
test['Rings'] = 0

cuda


In [5]:
class TabularDataset(Dataset):
    def __init__(self, data, target_column):
        self.data = data.drop(target_column, axis=1)
        
        # Convert data to float32 dtype for PyTorch and handle non-numeric issues
        for col in self.data.columns:
            self.data[col] = pd.to_numeric(self.data[col], errors='coerce')  # Coerce errors will convert non-numeric to NaN
        
        # Fill NaN values with the mean (or median, zero, etc.)
        self.data.fillna(self.data.mean(), inplace=True)
        
        self.targets = data[target_column].astype(float).values  # Ensure target is also float
        
        # Convert DataFrame to numpy array
        self.data = self.data.values.astype(np.float32)
        self.targets = self.targets.astype(np.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.from_numpy(self.data[idx])  # Now safely convert to tensor
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        return x, y

train, val = train_test_split(train, test_size=0.2)

train_loader = DataLoader(TabularDataset(train, 'Rings'), batch_size=64, shuffle=True, pin_memory=True)  # pin_memory for faster CPU->GPU transfers
val_loader = DataLoader(TabularDataset(val, 'Rings'), batch_size=64, shuffle=False, pin_memory=True)
test_loader = DataLoader(TabularDataset(test, 'Rings'), batch_size=64, shuffle=False, pin_memory=True)

In [6]:
class CNNRegressor(nn.Module):
    def __init__(self, num_features):
        super(CNNRegressor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=32, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
        self.conv4 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=1)
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = x.unsqueeze(2)  # Reshape x to [batch_size, num_features, 1] for Conv1D
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Number of features excluding the target
num_features = train.shape[1] - 1
model = CNNRegressor(num_features=num_features).to(device)


In [7]:
def rmsle_loss(y_pred, y_true):
    return torch.sqrt(F.mse_loss(torch.log1p(y_pred), torch.log1p(y_true)))

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = rmsle_loss(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.1308235079050064


In [9]:
with torch.no_grad():
    model.eval()
    val_loss = []
    for inputs, targets in val_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = rmsle_loss(outputs.squeeze(), targets)
        val_loss.append(loss.item())
    
    print(f'Validation Loss: {np.mean(val_loss)}')

Validation Loss: 0.16169371163925608


In [10]:
with torch.no_grad():
    model.eval()
    predictions = []
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.squeeze().cpu().numpy())
        
# save to csv 

submission = pd.DataFrame({'id': test.index, 'Rings': predictions})
submission.to_csv('submissions/cnn1.csv', index=False)